# Learning promtp engineering from GPT-4
### notebook version

raul.arrabales | Dec. '23


## Install required packages

In [1]:
# Already satisfied in my container
# ! pip install python-dotenv
# ! pip install tiktoken
# ! pip install openai
# ! pip install panel

## Connect to my data folder

In [5]:
# Using my local container data. 
# Use the following to use GDrive: 

# Mount my drive folder
# Only for Google Colab
# from google.colab import drive
# drive.mount('/content/drive')

## Libs

In [1]:
import os
import openai
import tiktoken
from dotenv import load_dotenv, find_dotenv
import json
from pprint import pprint

## Setup

In [2]:
# _ = load_dotenv('/content/drive/My Drive/Colab Notebooks/.env') # read drive colab folder .env file
_ = load_dotenv('myenv.txt')

openai.api_key  = os.environ['OPENAI_API_KEY']

## OpenAI API Helper functions

In [3]:
''' *** get_completion(prompt, model, temp) ***
        -----------------------------------
Helper function to get a GPT 3.5 Turbo completion
Using the chat completion API
(https://platform.openai.com/docs/guides/gpt/chat-completions-api)

INPUT:
- prompt: user's prompt
- model: OpenAI model (GPT 3.5 Turbo by default)
- temp: model's temperature (0 by default)

OUTPUT:
- The instruct LLM response
'''
def get_completion(prompt, model="gpt-3.5-turbo", temp=0):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temp,
    )
    return response.choices[0].message["content"]

In [4]:
''' *** get_completion_from_messages(messages, model, temp) ***
        ---------------------------------------------------
Helper function to get a GPT 3.5 Turbo completion.
In this case, from the list of past messages.
Using the chat completion API
(https://platform.openai.com/docs/guides/gpt/chat-completions-api)

INPUT:
- messages: list of messages
- model: OpenAI model (GPT 3.5 Turbo by default)
- temp: model's temperature (0 by default)

OUTPUT:
- The instruct LLM response
'''
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temp=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temp,
    )
    return response.choices[0].message["content"]

In [13]:
''' *** chat_with_context(prompt) ***
        -----------------------------
Gets completion using context global variable as the list of messages. 
Updates the context with current messages. 

INPUT:
- prompt: user's prompt

OUTPUT:
- Last LLM's response
'''
def chat_with_context(prompt, model="gpt-3.5-turbo"):
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context, model=model)
    context.append({'role':'assistant', 'content':f"{response}"})
    return response

In [6]:
# From https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb 
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613"):
    """Return the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model in {
        "gpt-3.5-turbo-0613",
        "gpt-3.5-turbo-16k-0613",
        "gpt-4-0314",
        "gpt-4-32k-0314",
        "gpt-4-0613",
        "gpt-4-32k-0613",
        }:
        tokens_per_message = 3
        tokens_per_name = 1
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif "gpt-3.5-turbo" in model:
        print("Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0613.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613")
    elif "gpt-4" in model:
        print("Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.")
        return num_tokens_from_messages(messages, model="gpt-4-0613")
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens."""
        )
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens

# Prompt Engineering

## First conversation

In [20]:
# Starting with an empty context
context = []

### Getting responses from GPT-4

In [21]:
system_role_prompt = f"""
You are an expert in LLMs and prompt engineering.
"""
# List of messages, starting with system's role
context.append({'role':'system', 'content':system_role_prompt})

In [22]:
prompt_0 = f"""
What is prompt engineering?. 
"""
context.append({'role':'user', 'content':f"{prompt_0}"})

In [23]:
response_0 = get_completion_from_messages(context, model='gpt-4-1106-preview')

In [24]:
print(response_0)

Prompt engineering is the process of carefully crafting inputs (prompts) to elicit desired outputs from language models, particularly large language models (LLMs) like OpenAI's GPT-3 or GPT-4, Google's BERT, or other AI systems. It involves understanding how the model interprets and responds to different types of language and structuring prompts to guide the model towards generating useful, accurate, and contextually appropriate responses.

Prompt engineering is both an art and a science, as it requires a blend of creativity, linguistic skill, and technical understanding of the model's capabilities and limitations. Effective prompt engineering can significantly enhance the performance of a language model, making it more applicable to a wide range of tasks such as content creation, code generation, information retrieval, conversation, and more.

Key aspects of prompt engineering include:

1. **Clarity**: Ensuring the prompt is clear and unambiguous to avoid confusion.
2. **Context**: Pr

In [25]:
prompt_1 = f"""
What is meta prompting?
"""
context.append({'role':'user', 'content':f"{prompt_1}"})

In [26]:
response_1 = get_completion_from_messages(context, model='gpt-4-1106-preview')

In [27]:
print(response_1)

Prompt engineering is the process of carefully crafting inputs (prompts) for language models, like GPT-3 or other large language models (LLMs), to elicit the best possible outputs. It involves understanding how the model interprets and responds to different types of prompts, and using this knowledge to design prompts that guide the model towards producing the desired text.

Prompt engineering can include:

1. **Precision in Language**: Using specific and clear language to reduce ambiguity in the prompt.
2. **Contextual Information**: Providing sufficient background information to help the model understand the context of the request.
3. **Instructional Prompts**: Giving direct instructions to the model about what kind of information or response is expected.
4. **Examples**: Including examples in the prompt to illustrate the type of response desired.
5. **Iterative Refinement**: Adjusting prompts based on the outputs received to improve the quality of future responses.

Meta prompting, o

In [28]:
prompt_2 = f"""
Can you give me an example of meta prompting applied to an insurance use case where I need the LLM \
to determine to what extent a specific claim is covered by the particular insurance policy conditions \
of the user? (I can include in the prompt both the claim description and policy conditions)
"""
context.append({'role':'user', 'content':f"{prompt_2}"})

In [29]:
response_2 = get_completion_from_messages(context, model='gpt-4-1106-preview')

In [30]:
print(response_2)

Meta prompting involves creating prompts that are self-referential or that instruct the language model to reflect on its own processes, capabilities, or the nature of the task it's being asked to perform. It's a way to guide the model to better understand the context or the meta-information about the task at hand.

In the insurance use case you've described, meta prompting could be used to help the language model understand that it needs to interpret the insurance policy conditions and apply them to the specific claim description to determine coverage. Here's an example of how you might structure such a meta prompt:

---

**Prompt:**

"Language model, you are to act as an assistant to an insurance claims adjuster. Your task is to analyze the text of an insurance policy and a description of a claim to determine the extent of coverage. You should consider yourself an expert system that can parse legal and insurance-related language, identify relevant clauses, and apply them to specific s